In [1]:
pip install datasets transformers torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
!mkdir -p ~/.kaggle
!echo '{"wandb_api_key": "5af3f6dab5b4be7bfb38dadf6554a3d40a09ada9"}' > ~/.kaggle/secrets.json
!chmod 600 ~/.kaggle/secrets.json


In [3]:
import json
import wandb

# Load API Key từ Kaggle Secrets
with open("/root/.kaggle/secrets.json", "r") as f:
    secrets = json.load(f)
    wandb_api_key = secrets["wandb_api_key"]

# Đăng nhập vào W&B
wandb.login(key=wandb_api_key)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: truongminhphuc08102005 (truongminhphuc08102005-hanoi-university-of-science-and-t). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForMultipleChoice, Trainer, TrainingArguments
from datasets import load_dataset
from tqdm import tqdm

dataset = load_dataset("allenai/qasc")


def map_answer_to_index(example):
    answer_key = example['answerKey']
    # Kiểm tra xem answer_key có phải là chuỗi rỗng không
    if answer_key:
        index = ord(answer_key) - ord('A')  # Chuyển A->0, B->1, ..., H->7
    else:
        index = -1  # Hoặc một giá trị mặc định khác khi answer_key là chuỗi rỗng
    return {'label': index}

dataset = dataset.map(map_answer_to_index)

# ### Bước 3: Tokenize dữ liệu
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

README.md:   0%|          | 0.00/7.54k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.97M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/224k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8134 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    questions = examples['question']
    choices = examples['choices']

    # Lặp lại question cho mỗi choice (8 lần mỗi câu hỏi)
    questions = [[q] * 8 for q in questions]

    # Truy cập danh sách các lựa chọn từ choice_dict['text']
    choices = [choice_dict['text'] for choice_dict in choices]

    # Flatten danh sách để tokenize
    questions = [item for sublist in questions for item in sublist]
    choices = [item for sublist in choices for item in sublist]

    # Tokenize các cặp (question, choice)
    inputs = tokenizer(
        questions,
        choices,
        truncation=True,
        padding='max_length',
        max_length=256
    )

    # Reshape lại thành batch_size x 8
    input_ids = [inputs['input_ids'][i:i+8] for i in range(0, len(inputs['input_ids']), 8)]
    attention_mask = [inputs['attention_mask'][i:i+8] for i in range(0, len(inputs['attention_mask']), 8)]
    token_type_ids = [inputs['token_type_ids'][i:i+8] for i in range(0, len(inputs['token_type_ids']), 8)]

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'token_type_ids': token_type_ids,
        'labels': examples['label']
    }
# Áp dụng preprocess với tqdm
encoded_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/8134 [00:00<?, ? examples/s]

Map:   0%|          | 0/920 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

In [7]:

model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

def compute_metrics(eval_pred):

    logits, labels = eval_pred
    logits = torch.tensor(logits)  # Chuyển logits thành tensor
    labels = torch.tensor(labels)  # Chuyển labels thành tensor

    # Sắp xếp các lựa chọn theo điểm số (logits) giảm dần
    _, indices = torch.topk(logits, k=8, dim=1)  # indices: top-8 indices theo logits

    # Khởi tạo các biến để tính metric
    precision_at_k = {k: 0.0 for k in [1, 3, 5]}
    recall_at_k = {k: 0.0 for k in [1, 3, 5]}
    mrr = 0.0
    total = len(labels)

    for i in range(total):
        true_label = labels[i].item()  # Nhãn đúng (chỉ số từ 0-7)
        ranked_indices = indices[i].tolist()  # Danh sách các chỉ số được xếp hạng

        # Tính MRR
        rank = ranked_indices.index(true_label) + 1 if true_label in ranked_indices else 0
        if rank > 0:
            mrr += 1.0 / rank

        # Tính Precision@k và Recall@k
        for k in [1, 3, 5]:
            top_k_indices = ranked_indices[:k]
            if true_label in top_k_indices:
                precision_at_k[k] += 1.0  # Đáp án đúng nằm trong top-k
                recall_at_k[k] += 1.0     # Đáp án đúng được tìm thấy trong top-k

    mrr /= total
    for k in [1, 3, 5]:
        precision_at_k[k] /= total
        recall_at_k[k] /= total

    return {
        "precision@1": precision_at_k[1],
        "precision@3": precision_at_k[3],
        "precision@5": precision_at_k[5],
        "recall@1": recall_at_k[1],
        "recall@3": recall_at_k[3],
        "recall@5": recall_at_k[5],
        "mrr": mrr
    }

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250321_014931-layz6j7c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./results
wandb: ⭐️ View project at https://wandb.ai/truongminhphuc08102005-hanoi-university-of-science-and-t/huggingface
wandb: 🚀 View run at https://wandb.ai/truongminhphuc08102005-hanoi-university-of-science-and-t/huggingface/runs/layz6j7c
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather alon

Epoch,Training Loss,Validation Loss,Precision@1,Precision@3,Precision@5,Recall@1,Recall@3,Recall@5,Mrr
1,1.703100,1.818910,0.313175,0.680346,0.865011,0.313175,0.680346,0.865011,0.533992
2,1.231500,2.044985,0.308855,0.676026,0.869330,0.308855,0.676026,0.869330,0.533992
3,0.877200,2.348374,0.314255,0.683585,0.866091,0.314255,0.683585,0.866091,0.537292


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=3051, training_loss=1.3113424275516097, metrics={'train_runtime': 4812.6125, 'train_samples_per_second': 5.07, 'train_steps_per_second': 0.634, 'total_flos': 2.5681513305563136e+16, 'train_loss': 1.3113424275516097, 'epoch': 3.0})

In [9]:
# Define a directory in Kaggle to save the fine-tuned model and tokenizer
save_directory = "/kaggle/working/BERT-QASC"

# Save the fine-tuned model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to /kaggle/working/BERT-QASC
